## User Management (Keycloak)
Interaction with EOEPCA endpoint using python Demo Client.

In [2]:
import utils.DemoKeycloakClient as Client
import jwt
import json
import requests
from keycloak import KeycloakPostError

base_domain = "develop.eoepca.org"
platform_domain = f"keycloak.{base_domain}"
server_url = f"https://{platform_domain}"
realm = "master"

## Client
We instantiate an object to interact with the platform. The object dynamically registers two Keycloak clients, one with admin privileges, another to act as a resource server to demo resources protection using UMA (User Managed Access).

In [ ]:
keycloak = Client.DemoKeycloakClient(server_url, realm, "admin", "admin_Abcd1234#")

## User Authentication
User authenticates and the client receives an ID Token (JWT) that represents the user, and is used to identify the user in UMA authorization flows.

#### Create Users
Create two users, an Eric and Alice both with user role.

In [ ]:
eric_id = keycloak.create_user("eric", "eric", ["user"])
alice_id = keycloak.create_user("alice", "alice", ["user"])

#### Inspect Eric User Token

In [ ]:
token = keycloak.get_user_token("eric", "eric")
print("Eric token:\n" + json.dumps(token, indent = 2))
eric_access_token = token["access_token"]
jwt_header = jwt.get_unverified_header(eric_access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(eric_access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

#### Inspect Alice User Token

In [ ]:
token = keycloak.get_user_token("alice", "alice")
print("Eric token:\n" + json.dumps(token, indent = 2))
alice_access_token = token["access_token"]
jwt_header = jwt.get_unverified_header(alice_access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(alice_access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

The ID Token (JWT) identifies the user via user_name / sub (Subject) fields, and the client via the aud (Audience) field. The JWT is signed and can be verified, using the kid (Key ID) field, via the JWKS endpoint of the Authorization Server.

## Protect resources
Access a protected resources using UMA flow

#### Assign premium role to Eric
Roles are used to define policies that will protect resources based on roles - Role-based access control (RBAC)

In [ ]:
keycloak.create_realm_role('user-premium')
keycloak.assign_realm_role_to_user(eric_id, 'user-premium')

#### Register resources
Register protected resources

In [ ]:
resources = [
    {
        "name": "Default Resource",
        "uri": "/*"
    },
    {
        'name': 'Protected Resource',
        'icon_uri': '/protected/*',
    },
    {
        "name": "Premium Resource",
        "uri": "/protected/premium/*"
    }
]
keycloak.register_resources(resources)

Right now, resources are protected by a default policy, which grants access to users within the realm.
Let's see how Keycloak protects resources using role based and user based policies.

#### Register policies
Register role based and user based policies

In [ ]:
keycloak.register_role_policy("Only Premium User Policy", "user-premium")
keycloak.register_user_policy("Only Eric User Policy", [eric_id])

#### Register resource permissions
Resources permissions are set by assigning policies to resources.

In [ ]:
permissions = [
    {
        "name": "Default Resource Permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Default Resource"
        ],
        "policies": [
            "Default Policy"
        ]
    },
    {
        "name": "Premium Resource Permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Premium Resource"
        ],
        "policies": [
            "Only Premium User Policy"
        ]
    },
    {
        "name": "Protected Resource Permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Protected Resource"
        ],
        "policies": [
            "Only User Policy"
        ]

    }
]
keycloak.assign_resources_permissions(permissions)

#### Get PAT (Protection API token)
PAT (Protection API token) is used to access Keycloak's Protection API, which manages resources and policies.

In [ ]:
pat = keycloak.generate_protection_pat()
print(json.dumps(pat, indent=2))
access_token = pat['access_token']

### Get Resource Id
Get Premium Resource id

In [ ]:
# it's possible to query resources by many fields, including name and uri
protected_resource_id = keycloak.get_resource_id(name="Premium Resource")[0]
print(protected_resource_id)
protected_resource_id = keycloak.get_resource_id(uri="/protected/premium/*")[0]
print(protected_resource_id)

#### Get UMA access token for premium resource for both Eric and Alice

In [ ]:
uma_ticket = keycloak.create_permission_ticket(resources=[protected_resource_id])['ticket']
print('UMA ticket for resource ' + protected_resource_id + ':\n' + uma_ticket)
eric_access_token = keycloak.get_user_token("eric", "eric")['access_token']
eric_rpt = keycloak.get_rpt(eric_access_token, uma_ticket)['access_token']
print('Eric RPT:\n' + str(eric_rpt))

In [ ]:
uma_ticket = keycloak.create_permission_ticket(resources=[protected_resource_id])['ticket']
print('UMA ticket for resource ' + protected_resource_id + ':\n' + uma_ticket)
alice_access_token = keycloak.get_user_token("alice", "alice")['access_token']
try:
    alice_uma_access_token = keycloak.get_rpt(alice_access_token, uma_ticket)['access_token']
except KeycloakPostError as e:
    print(str(e))

Trying to get a UMA token for Alice results in a 403 Forbidden Error. The reason being Alice is not allowed to access the `/protected/premium/*` resource because it's protected by a `user-premium` role which Alice does not have.

#### Use Eric UMA access token to access the premium resource

In [ ]:
headers = {
    "cache-control": "no-cache",
    "Authorization": "Bearer " + eric_rpt
}
premium_resource_url = "http://localhost:8080/protected/premium"
print('GET ' + premium_resource_url)
response = requests.get(premium_resource_url, headers=headers, verify=False)
print(str(response.status_code))

The expected result is 200 (or 404 if the demo app isn't running), meaning a successfuly access to the protected resource by Eric.

#### Identity-API Get Resources

In [1]:
import utils.DemoKeycloakClient as Client
import jwt
import json
import requests
from keycloak import KeycloakPostError

base_domain = "develop.eoepca.org"
platform_domain = f"keycloak.{base_domain}"
server_url = f"https://{platform_domain}"
realm = "master"
client_id = "gatekeeper"
identity_api_url = 'http://localhost:5566'#"https://identity.api.develop.eoepca.org"
get_resources_url = identity_api_url +"/"+ client_id + "/resources"
print("GET " + get_resources_url)
response = requests.get(get_resources_url)
try:
    print(response.json())
except:
    print(response)


GET http://localhost:5566/gatekeeper/resources
[{'_id': 'b4707e1f-2383-4c5e-bead-f83e3d34c3ed', 'attributes': {}, 'displayName': '', 'icon_uri': '', 'name': 'api-customers', 'owner': {'id': '844967e2-15e1-4e1b-96e5-e0effc8f1737', 'name': 'gatekeeper'}, 'ownerManagedAccess': False, 'scopes': [{'iconUri': '', 'id': '5869991a-2dbc-418f-ba15-2be087a9de50', 'name': 'view'}], 'uris': ['/api/customers']}, {'_id': '91836407-6c3a-4f45-8f14-c04358bc43e0', 'attributes': {}, 'name': 'Default Resource', 'owner': {'id': '844967e2-15e1-4e1b-96e5-e0effc8f1737', 'name': 'gatekeeper'}, 'ownerManagedAccess': False, 'type': 'urn:gatekeeper:resources:default', 'uris': ['/*']}, {'_id': '5bd655ec-2575-406e-aa08-28b1bd25f476', 'attributes': {}, 'name': 'Default Resource 5', 'owner': {'id': '844967e2-15e1-4e1b-96e5-e0effc8f1737', 'name': 'gatekeeper'}, 'ownerManagedAccess': False, 'type': 'urn:gatekeeper:resources:default', 'uris': ['/default5/*']}, {'_id': '6593ac93-858a-42bc-93ec-9e471787b672', 'attributes':

#### Identity-API Get resource by id

In [8]:

response = requests.get(get_resources_url)
resource_id = response.json()[0]["_id"]
print(resource_id)
get_resource_by_id_url = identity_api_url + "/resources/" + resource_id
response = requests.get(get_resource_by_id_url)
print(response.text)

api-customers
Unknown server error


#### Identity-API Register resource

In [29]:
client_id = "dummy-service"
post_resource_url = identity_api_url +"/"+ client_id +"/resources"
data = {
    'attributes': {}, 
    'name': 'Default Resource 4',
    'ownerManagedAccess': False, 
    'resource_scopes': [], 
    'type': 'urn:gatekeeper:resources:default', 
    'uris': ['/default4/*']
    }
response = requests.post(post_resource_url, json=data,)
print("Response code: ",str(response.status_code))
print("Resource added: ",response.json())

Response code:  200
Resource added:  {'_id': 'd7bf8f90-9a8b-4bee-b1c0-f920ed0e30e9', 'attributes': {}, 'name': 'Default Resource 4', 'owner': {'id': '337ea293-e5ea-4d67-bb70-1f89fd774c39', 'name': 'dummy-service'}, 'ownerManagedAccess': False, 'type': 'urn:gatekeeper:resources:default', 'uris': ['/default4/*']}


#### Identity-API Update resource

In [30]:
# get resource_id from the get_resources endpoint
resource_id = "d7bf8f90-9a8b-4bee-b1c0-f920ed0e30e9"
udpate_resource_url = identity_api_url +"/"+ client_id + "/resources/" + resource_id
data = {
  "attributes": {},
  "name": "Default Resource 5",
  "owner": {
    "id": "844967e2-15e1-4e1b-96e5-e0effc8f1737",
    "name": "dummy-service"
  },
  "ownerManagedAccess": False,
  "type": "urn:gatekeeper:resources:default",
  "uris": [
    "/default5/*"
  ]
}
response = requests.put(udpate_resource_url, json=data,)
print("Response code: ",str(response.status_code))

Response code:  500


#### Identity-API delete resource

In [31]:
# get resource_id from the get_resources endpoint
resource_id = "d7bf8f90-9a8b-4bee-b1c0-f920ed0e30e9"
delete_resource_url = identity_api_url +"/"+ client_id + "/resources/" + resource_id

response = requests.delete(delete_resource_url)
print("Response code: ",str(response.status_code))


Response code:  200


#### Identity-API Get client Policies

In [9]:
get_policies_url = identity_api_url +"/"+ client_id +  "/policies"
response = requests.get(get_policies_url)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())


Response code:  200
Response text:  [{'config': {'users': '["3b9ada08-7a96-4078-86fc-43c61248968e"]'}, 'decisionStrategy': 'UNANIMOUS', 'description': '', 'id': '57d4a363-6b40-4dec-93e9-a46a1a8e492f', 'logic': 'POSITIVE', 'name': 'admin_policy', 'type': 'user'}, {'config': {}, 'decisionStrategy': 'UNANIMOUS', 'description': 'despriction3', 'id': 'c838874b-f3a2-459d-90ef-8f94f565f9f1', 'logic': 'POSITIVE', 'name': 'aggregated policy 3', 'type': 'aggregate'}, {'config': {'code': '// by default, grants any permission associated with this policy\n$evaluation.grant();\n'}, 'decisionStrategy': 'AFFIRMATIVE', 'description': 'A policy that grants access only for users within this realm', 'id': 'cb607685-db58-4adb-a712-ba96bfa9e164', 'logic': 'POSITIVE', 'name': 'Default Policy', 'type': 'js'}, {'config': {'groups': '[{"id":"782dcbe6-40d1-4ebc-9c05-0b4e0829a141","extendChildren":false}]', 'groupsClaim': 'groups claim 1'}, 'decisionStrategy': 'UNANIMOUS', 'description': '', 'id': '58797dc5-c8ad-

#### Identity-API create client Policy

In [43]:
create_client_policies_url = identity_api_url +"/"+ client_id + "/policies/client"
data={
        "type": "client",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "name": "My Policy 2"
}
response = requests.post(create_client_policies_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  {'clients': ['gatekeeper'], 'decisionStrategy': 'UNANIMOUS', 'id': '78e92563-22ec-4716-9c90-3c3fff45fa35', 'logic': 'POSITIVE', 'name': 'My Policy 2', 'type': 'client'}


#### Identity-API Create Aggregated policy

In [47]:
create_aggregated_policies_url = identity_api_url +"/"+ client_id + "/policies/aggregated"
data = {
            "type": "aggregate",
            "logic": "POSITIVE",
            "decisionStrategy": "UNANIMOUS",
            "name": "aggregated policy 3",
            "policies": ["My Policy 2"],
            "description": "despriction3"
        }
response = requests.post(create_aggregated_policies_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  {'decisionStrategy': 'UNANIMOUS', 'description': 'despriction3', 'id': 'c838874b-f3a2-459d-90ef-8f94f565f9f1', 'logic': 'POSITIVE', 'name': 'aggregated policy 3', 'policies': ['78e92563-22ec-4716-9c90-3c3fff45fa35'], 'type': 'aggregate'}


#### Identity-API Create scope policy

In [48]:
create_scope_policies_url = identity_api_url +"/"+ client_id + "/policies/scope"
data = {
        "type": "client-scope",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "name": "My Policy 3"
}
response = requests.post(create_scope_policies_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  {'decisionStrategy': 'UNANIMOUS', 'id': 'ff60132e-58fc-49a9-bb2a-376ce442b501', 'logic': 'POSITIVE', 'name': 'My Policy 3', 'owner': 'gatekeeper', 'type': 'client-scope'}


#### Identity-API Create group policy

In [50]:
create_group_policies_url = identity_api_url +"/"+ client_id + "/policies/group"
data = {
            "type": "group",
            "logic": "POSITIVE",
            "decisionStrategy": "UNANIMOUS",
            "name": "group policy 1",
            "groups": [{"id": "group1", "path": "group1"}],
            "groupsClaim": "groups claim 1",
            "description": ""
        }
response = requests.post(create_group_policies_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  {'decisionStrategy': 'UNANIMOUS', 'description': '', 'groups': [{'extendChildren': False, 'id': '782dcbe6-40d1-4ebc-9c05-0b4e0829a141'}], 'groupsClaim': 'groups claim 1', 'id': '58797dc5-c8ad-4ea0-a06b-d34fd8de6cc5', 'logic': 'POSITIVE', 'name': 'group policy 1', 'type': 'group'}


#### Identity-API - Create regex Policy

In [52]:
create_regex_policies_url = identity_api_url +"/"+ client_id + "/policies/regex"
data = {
            "type": "regex",
            "logic": "POSITIVE",
            "decisionStrategy": "UNANIMOUS",
            "name": "regex policy 1",
            "pattern": "123",
            "targetClaim": "target1",
            "description": "description1"
        }
response = requests.post(create_regex_policies_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  {'decisionStrategy': 'UNANIMOUS', 'description': 'description1', 'id': '24656f8a-7004-4b5b-95ba-3b05998458e0', 'logic': 'POSITIVE', 'name': 'regex policy 1', 'pattern': '123', 'targetClaim': 'target1', 'type': 'regex'}


#### Identity-API Create role policy

In [57]:
create_role_policies_url = identity_api_url +"/"+ client_id + "/policies/role"
data = {
            "type": "role",
            "logic": "POSITIVE",
            "decisionStrategy": "UNANIMOUS",
            "name": "role policy 1",
            "roles": [
                {
                    "id": "b80601c3-9e0f-40fd-a03c-34cd759bfc63",
                    "required": False
                } 
            ]
        }
response = requests.post(create_role_policies_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  {'decisionStrategy': 'UNANIMOUS', 'id': 'a3ed3ec0-83e2-4df4-a57b-09e2b34a7a36', 'logic': 'POSITIVE', 'name': 'role policy 1', 'roles': [{'id': 'b80601c3-9e0f-40fd-a03c-34cd759bfc63', 'required': False}], 'type': 'role'}


#### Identity-API create time policy

In [61]:
create_time_policies_url = identity_api_url +"/"+ client_id + "/policies/time"
data = {
            "type": "time",
            "logic": "POSITIVE",
            "decisionStrategy": "UNANIMOUS",
            "name": "time policy 1",
            "description": "",
            "year": 2023
        }
# time can be one of:
        # "notAfter":"1970-01-01 00:00:00"
        # "notBefore":"1970-01-01 00:00:00"
        # "dayMonth":<day-of-month>
        # "dayMonthEnd":<day-of-month>
        # "month":<month>
        # "monthEnd":<month>
        # "year":<year>
        # "yearEnd":<year>
        # "hour":<hour>
        # "hourEnd":<hour>
        # "minute":<minute>
        # "minuteEnd":<minute>
response = requests.post(create_time_policies_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  {'decisionStrategy': 'UNANIMOUS', 'description': '', 'id': 'f9031d6a-9238-413d-aade-9e1e2353820f', 'logic': 'POSITIVE', 'name': 'time policy 1', 'type': 'time', 'year': '2023'}


#### Identity-API Create user policy

In [62]:
create_user_policies_url = identity_api_url +"/"+ client_id + "/policies/user"
data = {
            "type": "user",
            "logic": "POSITIVE",
            "decisionStrategy": "UNANIMOUS",
            "name": "user policy 1",
            "users": "3b9ada08-7a96-4078-86fc-43c61248968e",
            "description": "descriptitirnnce"
        }
response = requests.post(create_user_policies_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  {'decisionStrategy': 'UNANIMOUS', 'description': 'descriptitirnnce', 'id': '8e90c31f-759a-4679-86c5-56cad118f12e', 'logic': 'POSITIVE', 'name': 'user policy 1', 'type': 'user', 'users': ['3b9ada08-7a96-4078-86fc-43c61248968e']}


#### Identity-API Update policies

In [67]:
policy_id = "8e90c31f-759a-4679-86c5-56cad118f12e"
update_policies_url = identity_api_url + "/"+ client_id +"/policies/" + policy_id
data = {
            "type": "user",
            "logic": "POSITIVE",
            "decisionStrategy": "UNANIMOUS",
            "name": "user policy 12",
            "description": "descriptitirnnce222"
        }
response = requests.put(update_policies_url,json=data)
print("Response code: ",str(response.status_code))

Response code:  200


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

#### Identity-API delete policies

In [68]:
policy_id = "8e90c31f-759a-4679-86c5-56cad118f12e"
delete_policies_url = identity_api_url + "/"+ client_id +"/policies/" + policy_id
response = requests.put(delete_policies_url)
print("Response code: ",str(response.status_code))

Response code:  200


#### Identity-API get client permissions

In [3]:
get_client_permissions_url = identity_api_url + "/"+ client_id +"/permissions"
response = requests.get(get_client_permissions_url)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())


Response code:  200
Response text:  [{'decisionStrategy': 'UNANIMOUS', 'description': 'A permission that applies to the default resource type 1', 'id': '2193eed8-93e5-471f-bb67-a3726b333fb1', 'logic': 'POSITIVE', 'name': 'Default Permission', 'resourceType': 'urn:gatekeeper:resources:default', 'type': 'resource'}, {'decisionStrategy': 'UNANIMOUS', 'id': 'aad1133f-cc7f-4b5e-9feb-553bc239dc5d', 'logic': 'POSITIVE', 'name': 'resource1permission', 'type': 'resource'}, {'decisionStrategy': 'UNANIMOUS', 'id': '9098614b-0964-4b74-ba0b-1f2b68544a70', 'logic': 'POSITIVE', 'name': 'resource2permission', 'type': 'resource'}, {'decisionStrategy': 'UNANIMOUS', 'id': 'addd7e41-0f94-4d85-bb0c-b6aa8df65186', 'logic': 'POSITIVE', 'name': 'resource3permission', 'type': 'resource'}, {'decisionStrategy': 'UNANIMOUS', 'description': '', 'id': '8c16b640-f9f3-4eef-b2a0-b7812aea1c66', 'logic': 'POSITIVE', 'name': 'resource-based-permission', 'type': 'resource'}, {'decisionStrategy': 'UNANIMOUS', 'description'

#### Identity-API get client management permissions

In [77]:
get_management_permissions_url = identity_api_url + "/"+ client_id +"/permissions/management"
response = requests.get(get_management_permissions_url)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())


Response code:  200
Response text:  {'enabled': False}


#### Identity-API get client resources permissions

In [78]:
get_resources_permissions_url = identity_api_url + "/"+ client_id +"/permissions/resources"
response = requests.get(get_resources_permissions_url)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())


Response code:  200
Response text:  [{'decisionStrategy': 'UNANIMOUS', 'description': 'A permission that applies to the default resource type', 'id': '2193eed8-93e5-471f-bb67-a3726b333fb1', 'logic': 'POSITIVE', 'name': 'Default Permission', 'resourceType': 'urn:gatekeeper:resources:default', 'type': 'resource'}, {'decisionStrategy': 'UNANIMOUS', 'description': '', 'id': '8c16b640-f9f3-4eef-b2a0-b7812aea1c66', 'logic': 'POSITIVE', 'name': 'resource-based-permission', 'type': 'resource'}]


#### Identity-API create client resources permissions

In [81]:
create_resources_permissions_url = identity_api_url + "/"+ client_id +"/permissions/resources"
data={
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "name": "Permission-Name 2",
        "resources": [
            "5bd655ec-2575-406e-aa08-28b1bd25f476"
        ],
        "policies": [
            "57d4a363-6b40-4dec-93e9-a46a1a8e492f"
        ]
}
response = requests.get(get_resources_permissions_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  [{'decisionStrategy': 'UNANIMOUS', 'description': 'A permission that applies to the default resource type', 'id': '2193eed8-93e5-471f-bb67-a3726b333fb1', 'logic': 'POSITIVE', 'name': 'Default Permission', 'resourceType': 'urn:gatekeeper:resources:default', 'type': 'resource'}, {'decisionStrategy': 'UNANIMOUS', 'description': '', 'id': '8c16b640-f9f3-4eef-b2a0-b7812aea1c66', 'logic': 'POSITIVE', 'name': 'resource-based-permission', 'type': 'resource'}]


#### Identity-API update client management permissions

In [82]:
update_management_permissions_url = identity_api_url + "/"+ client_id +"/permissions/management"
data={
    "enabled": False
}
response = requests.put(update_management_permissions_url,json=data)
print("Response code: ",str(response.status_code))
print("Response text: ",response.json())

Response code:  200
Response text:  {'enabled': False}


#### Identity-API update client resources permissions

In [85]:
permission_id = "2193eed8-93e5-471f-bb67-a3726b333fb1"
update_resources_permissions_url = identity_api_url + "/"+ client_id +"/permissions/resources/" + permission_id
data={
        "decisionStrategy": "UNANIMOUS",
        "description": "A permission that applies to the default resource type 1",
        "logic": "POSITIVE",
        "name": "Default Permission",
        "resourceType": "urn:gatekeeper:resources:default",
        "type": "resource"
    }
response = requests.put(update_resources_permissions_url,json=data)
print("Response code: ",str(response.status_code))

Response code:  200


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

#### Identity-API Register and Protect a Resource

In [10]:
register_and_protect_url = identity_api_url + "/" + client_id +"/register-resources"
data = [{
            "resource":{
                "name": "resource1",
                "uris": ["/resource1/*"],
                "attributes": {},
                "resource_scopes": ["access"]
            },
            "permissions": {
                "user":["eric"]
            }
        },
        {
            "resource":{
                "name": "resource2",
                "uris": ["/resource2/*"],
                "attributes": {},
                "resource_scopes": ["access"]
            },
            "permissions": {
                "user":["alice"]
            }
        }]

response = requests.post(register_and_protect_url,json=data)
print("Response code: ",str(response.status_code))

Response code:  200


#### Identity-API Delete Resource and its policies and permissions

In [9]:
resource_name = "resource1"
delete_resources_url = identity_api_url + "/" + client_id +"/delete-resources/" + resource_name
response = requests.delete(delete_resources_url)
print(response.status_code)
print(response.text)

200
{}



#### Identity-API Create client, create and protect resources in one endpoint

In [20]:
import utils.DemoKeycloakClient as Client
import jwt
import json
import requests
from keycloak import KeycloakPostError
identity_api_url = 'http://localhost:5566'#"https://identity.api.develop.eoepca.org"
create_client_url = identity_api_url + '/create-client'
payload = {
           "protocol":"openid-connect",
           "clientId":"dummy-service3",
           "name":"",
           "description":"",
           "publicClient":False,
           "authorizationServicesEnabled":True,
           "serviceAccountsEnabled":True,
           "implicitFlowEnabled":False,
           "directAccessGrantsEnabled":True,
           "standardFlowEnabled":True,
           "frontchannelLogout":True,
           "attributes":{
               "saml_idp_initiated_sso_url_name":"",
               "oauth2.device.authorization.grant.enabled":False,
               "oidc.ciba.grant.enabled":False
               },
            "alwaysDisplayInConsole":False,
            "rootUrl":"",
            "baseUrl":"",
            "redirectUris":["*"],
            "resources":[{
                "resource":{
                    "name": "resource1",
                    "uris": ["/resource1/*"],
                    "attributes": {},
                    "resource_scopes": ["access"]
                },
                "permissions": {
                    "user":["eric"]
                }
            },
            {
                "resource":{
                    "name": "resource2",
                    "uris": ["/resource2/*"],
                    "attributes": {},
                    "resource_scopes": ["access"]
                },
                "permissions": {
                    "user":["alice"]
                }
            }]
}
response = requests.post(create_client_url, json=payload)
print(response.status_code)
print(response.text)

200
[
  {
    "_id": "810fd94f-4c7f-4f95-a262-c01cfaa6dc43",
    "attributes": {},
    "name": "resource1",
    "owner": {
      "id": "40473bde-0270-424a-a32b-088dd3f23c5a",
      "name": "dummy-service3"
    },
    "ownerManagedAccess": false,
    "scopes": [
      {
        "id": "5f7564d1-1479-475a-a955-08268b799aeb",
        "name": "access"
      }
    ],
    "uris": [
      "/resource1/*"
    ]
  },
  {
    "_id": "c352ab94-e81d-4d8d-91ff-87c590092d8c",
    "attributes": {},
    "name": "resource2",
    "owner": {
      "id": "40473bde-0270-424a-a32b-088dd3f23c5a",
      "name": "dummy-service3"
    },
    "ownerManagedAccess": false,
    "scopes": [
      {
        "id": "5f7564d1-1479-475a-a955-08268b799aeb",
        "name": "access"
      }
    ],
    "uris": [
      "/resource2/*"
    ]
  }
]

